# CP423: Assignment 1

Group Number: 5

Group Members: Abigail Lee (200469770), Kayleigh Habib (200370580) and Myisha Chaudhry (200591740)

In [1]:
# MIGHT NEED TO RUN THIS FOR THE NLTK
#pip install certifi

In [2]:
import certifi
import ssl
import os

os.environ["SSL_CERT_FILE"] = certifi.where()

In [3]:
# inputs
import os
import re

import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Question 1 : Text Preprocessing
a. Convert all text to lowercase

b. Tokenize the text using NLTK

c. Remove stop words using NLTK

d. Exclude special characters except alphanumeric characters

e. Eliminate singly occurring characters

f. Create a set of all the words

In [4]:
def text_preprocessing_str(query):

    # a. converting to lowercase
    lines = [x.lower() for x in query]

    # join all the lines into one string
    data = "".join(lines)

    # b. using the tokenization package to tokenize the words
    tokenization = word_tokenize(data)

    # remove duplicates - less words to do the next steps with
    newtoken = list(dict.fromkeys(tokenization))

    # c. remove any stop words from the NLTK package
    stop_words_lib = set(stopwords.words("english")) # choosing the english stopwords from the package

    # goes through each word, and checks if it is in the library - if it is not then we keep the word, if it is we don't take it
    stop_words = []
    for word in newtoken:
            if word not in stop_words_lib:
                 stop_words.append(word)

    # print(stop_words)

    # d. Remove special characters
    pattern = "[^a-z0-9]" # pattern NOT with any letter between a-z or any number including 0-9 (could also do A-Z but since we already made everything lower don't need to add that)
    alphanumerictxt = []
    # goes through each index in the list and if it matches the pattern it will put an empty string otherwise will preserve the value
    # strips the empty string afterwards
    for i in stop_words:
        alphanumerictxt.append(re.sub(pattern," ", i).strip())

    # e. eliminating any characters of length 1
    updated_list = []

    # goes through each index in the list and if the length is more than 1 will append to new list
    for i in alphanumerictxt:
        if len(i)>1:
            updated_list.append(i)

    # after all the other steps are completed check again to make sure no duplicated values
    final_list = list(dict.fromkeys(updated_list))


    return final_list

In [5]:
def text_preprocessing(file):

    # reading the file line by line
    lines = file.readlines()

    # a. converting to lowercase
    lines = [x.lower() for x in lines]

    # join all the lines into one string
    data = "".join(lines)

    final_list = text_preprocessing_str(data)

    #f
    return final_list


In [6]:
# import data files
def readfile_fun():
  # takes the names of the files in the folder and creates a list
  file_names = os.listdir("./data")
  # file_names

  # file id (key)
  counter = 1

  #dictionary1 - translates the ID which belong to the file
  dict_words = {}
  # dictionaty2 - file dictionary (id to file name)
  dict_files = {}

  # goes through each file in the list of file names
  for i in file_names:

      # if counter <= 2: #for checking purposes - can remove before submitting

        # gets the working directory (this may be different for everyone)
        file_path = os.getcwd() + "/data/" + i
        # opens the file in reading mode
        file = open(file_path, "r", encoding="utf-8", errors = "ignore" ) # added the encoding and errors as some files had different encoding detectors (note: with new line can add new param --> newline = '/r/n'))
        # calls the function written for question 1 of the assignment
        words = text_preprocessing(file)

        # updates the dictionaries
        dict_words[counter] = words # list of words in this file
        dict_files[counter] = i # list of all the file names

      # increment the counter
        counter+=1

      # closes the file
        file.close()

  print(dict_files)
  print(dict_words)

  return dict_words, dict_files

## Question 2 : Inverted Index Implementation
Implement an inverted index data structure for the preprocessed dataset


In [7]:
class HashInvertedIndex:

    # initialize hash table
    def __init__(self, capacity=100): # manually set the capacity
        self.capacity = capacity
        self.table = [[] for _ in range(capacity)]
        self.count = 0

    # finding the corresponding value for the key
    def _index(self, word):
        index = hash(word) % self.capacity
        return index

    # add key-value pair to the hash table
    def insert(self, key, value):

        # compute the hash index for the word
        hash_index = self._index(key)
        for i, kv in enumerate(self.table[hash_index]):
            k, v = kv
            if k == key:
                # add the value to the existing set of values
                v.add(value)
                # add the entry to the table
                self.table[hash_index][i] = (k,v)
                return
        self.table[hash_index].append((key, {value}))

    # retrieve the set of values for a given key, if not found, returns an empty set
    def retrieve(self, key):
        # compute the hash index for a given key
        hash_index = self._index(key)
        for k, v in self.table[hash_index]:
            # if the key if found, return the set
            if k == key:
                return v
        return set()

    # for testing purposes - shows us the string representation of the hash tables
    def __rep__(self):
        return str(self.table)


In [8]:
# build the inverted index - reads the documents and their corresponding words
def create_inverted_index():

    # initialize the hash table/inverted index
    inverted_index = HashInvertedIndex()

    # calls the function above
    # returns list of words in doc and list of docs
    words, files = readfile_fun()

    # fill in the inverted index
    for key, value in words.items():
        for word in value:
            inverted_index.insert(word, key) # inserting counter instead of doc name

    return inverted_index, files, words


In [22]:
# calls the function
inverted_index, file_list, word_list = create_inverted_index()

# prints out the index and the list of file names
# index to first list is hash to the word
for i in range(len(inverted_index.table)):
    #for j in i:
    # prints the hash buckets
    print(inverted_index.table[i])
    # prints the length of each bucket (right now around 400-500 - evenly distributed) - if we want smaller amounts in each bucket set bucket_size = 200
    #print(len(inverted_index.table[i]))

#print(file_list)


Output hidden; open in https://colab.research.google.com to view.

## Question 3: Query Support
Support the following queries, where x and y would be taken as input from the user

1. x OR y

2. x AND y

3. x AND NOT y

4. x OR NOT y


In [32]:
def query(sentence, operation, InvertedIndex, file):

  #set count of comparisons to 0 at beginning
  #set index to 0
  #set word to the firts word in the sentence
  #set final_list to the inverted index for the first word in the sentence

  comparisons = 0
  index = 0
  word = sentence[index]
  final_list = InvertedIndex.retrieve(word)

  #loop through array of operations
  while index < len(operation):

    #OR
    if operation[index] == "OR":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as UNION will put every value from the min list into the max list removing duplicates
      # using the final_list as the base, counting comparisons made (length of smaller list)
      comparisons += min(len(final_list),len(list1))

      # using the union function to combine the final_list with list1 in order to capture all documents with both words
      final_list = final_list.union(list1)
      # print(final_list)

    #AND
    elif operation[index] == "AND":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # looking at the minimum of the smallest list as INTERSECTION will keep matching values in the max list removing duplicates
      # using the final_list as the base, counting comparisons made (length of smaller list)
      comparisons += min(len(final_list),len(list1))

      final_list = final_list.intersection(list1)

    elif operation[index] == "AND NOT":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # to get the intersection

      # looking at the minimum of the smallest list, keeping count of the values compared to the base list
      # adding in the values in the intersection of both lists, as values are compared for items that exist in both

      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))

      # removing values that exist in both lists from the list of values for the first word (values that exist for word1 and not word2)
      final_list = final_list - (final_list.intersection(list1))

    elif operation[index] == "OR NOT":

      # set the next word in the sentence and retrieve the inverted index for that using the retrieve function from the class in Q2
      next_word = sentence[index+1]
      list1 = InvertedIndex.retrieve(next_word)

      # using the two inverted indexes to find where the first word exists or the second word does not
      # creating temp variable to store list of values where only second word is present
      temp = list1 - (list1.intersection(final_list))

      # looking at the minimum of the smallest list, keeping count of the values compared to the base list
      # adding in the values in the intersection of both lists, as values are compared for items that exist in both
      # adding in the values for comparisons done where only the second word is present

      comparisons += min(len(final_list),len(list1))
      comparisons += len(final_list.intersection(list1))
      comparisons += len(temp)

      # using all document names/values as a universal set, and removing document names/values where only the second word exists
      # final_list = set(file.values()) - temp
      final_list = set(file.keys()) - temp
      print(final_list)

    index+=1

  return final_list, comparisons



## Question 4: System Evaluation
Evaluate your system against the set of provided queries.

1. Query #1

- Input Sentence: "lion stood thoughtfully for a moment"

- Input operation sequence: [OR, OR, OR]

- Expected preprocessed query: "lion OR stood OR thoughtfully OR moment"

2. Query #2

- Input Sentence: "telephone, paved, roads"

- Input operation sequence: [OR NOT, AND NOT]

- Expected preprocessed query: "telephone OR NOT paved AND NOT roads


Output should include:
- The number of documents retrieved

- The minimum number of total comparisons made (only for merging the algorithm)

- The list of retrieved document names

In [33]:
text = input("Enter your string: ")
sentence_list = text_preprocessing_str(text)

operations = input("Enter the operations (in CAPS, seperated with commas, NO SPACES between the operations): ")
operations_list = operations.split(",")
print("Sentence: " ,sentence_list)
print("Operations list: ", operations_list)
match_list, comp = query(sentence_list, operations_list, inverted_index,file_list)


print("Number of Matched documents: " , len(match_list))
print("Minimum number of comparisons required: ", comp)
print("List of retreived document names: ")
for match in match_list:
     print(file_list[match])

Enter your string: hello world
Enter the operations (in CAPS, seperated with commas, NO SPACES between the operations): OR NOT
Sentence:  ['hello', 'world']
Operations list:  ['OR NOT']
{2, 3, 5, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 33, 37, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 53, 54, 55, 57, 58, 60, 61, 63, 64, 65, 67, 68, 70, 72, 73, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 87, 89, 90, 91, 93, 94, 95, 97, 98, 101, 103, 104, 105, 107, 108, 110, 112, 114, 115, 117, 118, 121, 122, 123, 124, 127, 128, 129, 130, 132, 133, 134, 135, 136, 137, 138, 140, 141, 143, 146, 148, 150, 152, 155, 157, 158, 159, 160, 163, 166, 167, 168, 169, 170, 171, 172, 174, 177, 178, 180, 182, 183, 184, 185, 186, 189, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 201, 202, 204, 206, 208, 209, 210, 211, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 228, 229, 230, 231, 232, 233, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249}
Number of Mat